# TODO

    -> #later function to run sextractor from jupyter 
    -> #later function to plot squares in an image centered in a given RA and DEC (params: size and color of point)
    -> #later function to load and preprocess correctly the output of sextractor
    -> function to select only objects within range (+- 0.01 RA and DEC) from positions of known LSBs
    -> function to plot these selected objects over the greyscale pic

# Imports

In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob
from config import IMAGES_DIR

In [2]:
files = glob.glob(IMAGES_DIR+"*.npz")
data = [np.load(file)['img'] for file in files][0]

In [3]:
img = np.load('data/images/SPLUS-s28s34_rgb.npz')['img']

In [4]:
mask = get_mask(img[:,:,0],pixel_threshold=250)

NameError: name 'get_mask' is not defined

In [5]:
LSBs_coords = pd.read_csv("data/LSBs_coordinates/SPLUS-s28s34.csv", usecols = ["Status","RA","DEC"])
detected_original,detected_masked = [pd.read_csv(f'data/results/{f}.csv', usecols=["RA","DEC"]) for f in ["original","masked"]]


FileNotFoundError: [Errno 2] No such file or directory: 'data/results/original.csv'

In [ ]:
def filter_region(coords:pd.DataFrame, 
                  lsb:pd.DataFrame, 
                  threshold:float = 0.01)->pd.DataFrame:
    """
    returns all objects in coords that are within a certain threshold from the objects in lsb
    """
    
    lsb_coords = zip(lsb.RA.to_list(),lsb.DEC.to_list())
    filter_conditions = [f"({RA-0.01} <= RA <= {RA+0.01} and {DEC-0.01} <= DEC <= {DEC+0.01})" for RA,DEC in lsb_coords]
    filter_query = " or ".join(filter_conditions)
    
    return coords.query(filter_query)

In [ ]:
orig_detected_filtered = filter_region(detected_original,LSBs_coords)
mask_detected_filtered = filter_region(detected_masked,LSBs_coords)

In [ ]:
plt.imsave('data/results/s28s34.png',arr=img[:,:,0] , cmap='gray_r')

In [ ]:
w = 11000/192

fig = plt.figure(frameon=False)
fig.set_size_inches(w,w)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
for row in orig_detected_filtered.values.tolist():
    RA,DEC = row
    position = coord2pix(wcs_obj,[RA, DEC])
    plt.scatter(position[0], position[1], marker = "+", c= "r", s = 1);
plt.imshow(img[:,:,0] , cmap='gray_r');
fig.savefig('data/results/original_detected.png', dpi=192)

In [ ]:
w = 11000/192

fig = plt.figure(frameon=False)
fig.set_size_inches(w,w)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
for row in mask_detected_filtered.values.tolist():
    RA,DEC = row
    position = coord2pix(wcs_obj,[RA, DEC])
    plt.scatter(position[0], position[1], marker = "+", c= "r", s = 1);
plt.imshow(img[:,:,0] , cmap='gray_r');
fig.savefig('data/results/masked_detected.png', dpi=192)

In [ ]:
marked_original = plt.imread('data/results/original_detected.png')
marked_masked = plt.imread('data/results/masked_detected.png')

In [ ]:
img.shape, marked_original.shape, marked_masked.shape

In [ ]:
fig,ax = plt.subplots(32,3,figsize=(30,10*32))
masked_masked = marked_masked*np.dstack([mask]*4)
for i,row in LSBs_coords.iterrows():
    RA,DEC,STATUS = row
    ax[i,0].imshow(getcutout(wcs_obj, img, [RA,DEC], size=100)[:,:,0],cmap='gray_r')
    ax[i,0].set_ylabel(STATUS, fontsize=25)
    ax[i,0].set_title("original", fontsize=25)
    ax[i,1].imshow(getcutout(wcs_obj, marked_original, [RA,DEC], size=100))
    ax[i,1].set_title("detected w/o mask", fontsize=25)
    ax[i,2].imshow(getcutout(wcs_obj, masked_masked, [RA,DEC], size=100))
    ax[i,2].set_title("detected with mask", fontsize=25)

fig.savefig('data/results/sumary_sextractor_minarea40.png')

In [ ]:
for row in LSBs_coords.iterrows():
    RA,DEC,*_ = row[1]
    print(RA,DEC)

In [ ]:
for row in detected_original.iterrows():
    RA,DEC = row[1]
    print(RA,DEC)

In [ ]:
.query("55.02 <= RA <= 55.03" and "-36.55 <= DEC <= -36.59")#.drop(columns=["index",""])

In [ ]:
from mask import get_mask
mask = get_mask(data[:,:,0],pixel_threshold=255)

In [ ]:
54.60513383,-36.89015933
55.02931567,-36.55989569
54.9069489,-36.55310976
54.38493552,-36.54482525
54.57997274,-36.06583223
54.655183,-35.91720684
54.10520922,-35.70303306
55.2788723,-35.69009573

In [ ]:
coord2pix(data_wcs,[54.3849355175621,-36.5448252522365])


In [ ]:
coord2pix(data_wcs,[55.0376, -35.6248])

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
#plt.scatter(positions[0], positions[1], marker = "s", s = 50);
plt.imshow(np.flipud(getcutout(np.flipud(data),coord2pix(data_wcs,[54.60513383,-36.89015933]), 200)));

In [ ]:
size = 50
pixy = int(8725.975)
pixx = int(3140.8179)

In [ ]:
import astropy.wcs as wcs
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from toolz import curry

In [ ]:
@curry
def coord2pix(wcs_obj,coords:iter)->tuple:
    """
    returns the pixel location given a pair RA, DEC
    """
    coords_t = SkyCoord(ra = coords[0]*u.degree, dec = coords[1]*u.degree)
    return wcs_obj.world_to_pixel(coords_t)

def getwcs(filepath:str):
    """
    returns the wcs object from a fits file
    """
    file = fits.open(filepath)
    return wcs.WCS(f[0].header)

@curry
def getcutout(wcs_obj,img:np.array,coords:iter, size:int)->np.array:
    """
    returns a cutout image centered in coords [RA,DEC]
    """
    im_coords = coord2pix(wcs_obj,coords)
    y,x = [int(coord) for coord in im_coords]
    return img[x - size:x + size,y - size:y + size]
    

In [ ]:
wcs_obj = getwcs('data/fits/SPLUS-s28s34_band_R.fits')


RA, DEC = 54.3849355175621,-36.5448252522365
coord2pix(data_wcs,[RA, DEC])

In [ ]:
positions = w.world_to_pixel(SkyCoord(ra = RA*u.degree, dec = DEC*u.degree))

In [ ]:
positions

In [ ]:
coordinates = pd.read_csv("data/LSBs_coordinates/SPLUS-s28s34.csv", usecols = ["Status","RA","DEC"])

In [ ]:
coordinates.apply(coord2pix(data))

In [ ]:
a = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 7, 0, 0, 0, 2, 0, 0, 0, 1],
              [0, 2, 1, 0, 0, 0, 0, 0, 3, 0],
              [0, 0, 0, 2, 0, 2, 1, 7, 2, 0],
              [0, 8, 0, 0, 0, 0, 0, 2, 0, 0],
              [2, 0, 0, 0, 0, 2, 0, 0, 0, 0],
              [0, 0, 0, 9, 0, 0, 0, 0, 0, 5],
              [0, 6, 0, 0, 1, 0, 0, 0, 1, 5],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
getcutout(a,[6,5],2)